In [28]:
# BEGIN SOLUTION
# Put all your imports here.

import pandas as pd
import numpy as np
import datetime as dt
import sqlalchemy as sq


## Loading and displaying the dataset Application Record [[1]](https://www.kaggle.com/rikdifos/credit-card-approval-prediction)

In [29]:

application_record = pd.read_csv('./CreditCardPrediction/application_record.csv')

display(application_record)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0
438553,6840222,F,N,N,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0
438554,6841878,F,N,N,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0
438555,6842765,F,N,Y,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,NaN,2.0


### Dropping columns as they do not have much of the role and impact in determining whether the person should be given a credit card or not. These columns include, 
 - CODE_GENDER
 - FLAG_OWN_CAR 
 - CNT_CHILDREN
 - NAME_EDUCATION_TYPE
 - NAME_FAMILY_STATUS
 - FLAG_WORK_PHONE
 - FLAG_PHONE 
 - FLAG_EMAIL
 - OCCUPATION_TYPE
 - CNT_FAM_MEMBERS

In [30]:
application_record = application_record.drop(['CODE_GENDER', 'FLAG_OWN_CAR', 'CNT_CHILDREN', 'NAME_EDUCATION_TYPE', 
                                                 'NAME_FAMILY_STATUS', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL',
                                                 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS'], axis = 1)

display(application_record)

,ID,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL
0,5008804,Y,427500.0,Working,Rented apartment,-12005,-4542,1
1,5008805,Y,427500.0,Working,Rented apartment,-12005,-4542,1
2,5008806,Y,112500.0,Working,House / apartment,-21474,-1134,1
3,5008808,Y,270000.0,Commercial associate,House / apartment,-19110,-3051,1
4,5008809,Y,270000.0,Commercial associate,House / apartment,-19110,-3051,1
...,...,...,...,...,...,...,...,...
438552,6840104,Y,135000.0,Pensioner,House / apartment,-22717,365243,1
438553,6840222,N,103500.0,Working,House / apartment,-15939,-3007,1
438554,6841878,N,54000.0,Commercial associate,With parents,-8169,-372,1
438555,6842765,Y,72000.0,Pensioner,House / apartment,-21673,365243,1


###  Here we are formatting the DAYS_BIRTH column to the DATE_OF_BIRTH of the person by formatting it. DAYS_BIRTH column count backwards from current day (0), -1 means yesterday. Similarly we format DAYS_EMPLOYED column with a Y and N flag with Y being an indicator that person is employed and N implying a person being unemployed [[2]](https://stackoverflow.com/questions/40104946/how-to-get-date-after-subtracting-days-in-pandas). 

### For the column DAYS_EMPLOYED if positive, it means the person is currently unemployed. We renamed the column FLAG_OWN_REALTY to OWN_PROPERTY making the naming for understandable that the person owns a property. Further after making changes with the help of DAYS_BIRTH and DAYS_EMPLOYED columns we drop them as they are not required further.


In [31]:
application_record['DATE_OF_BIRTH'] = pd.to_datetime('today').strftime("%d-%m-%Y")
application_record['DATE_OF_BIRTH'] = application_record['DATE_OF_BIRTH'].astype('datetime64')

application_record['DATE_OF_BIRTH'] = application_record['DATE_OF_BIRTH'] + application_record.DAYS_BIRTH.apply(lambda x: pd.Timedelta(x, unit='D'))

application_record['EMPLOYMENT_STATUS'] = np.where(application_record['DAYS_EMPLOYED'] < 0, 'Y', 'N')

application_record = application_record.rename(columns={'FLAG_OWN_REALTY': 'OWN_PROPERTY'})

application_record_data = application_record.drop(['DAYS_BIRTH', 'DAYS_EMPLOYED'], axis = 1)

display(application_record_data)

,ID,OWN_PROPERTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_HOUSING_TYPE,FLAG_MOBIL,DATE_OF_BIRTH,EMPLOYMENT_STATUS
0,5008804,Y,427500.0,Working,Rented apartment,1,1988-03-01,Y
1,5008805,Y,427500.0,Working,Rented apartment,1,1988-03-01,Y
2,5008806,Y,112500.0,Working,House / apartment,1,1962-03-29,Y
3,5008808,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
4,5008809,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
...,...,...,...,...,...,...,...,...
438552,6840104,Y,135000.0,Pensioner,House / apartment,1,1958-11-02,N
438553,6840222,N,103500.0,Working,House / apartment,1,1977-05-24,Y
438554,6841878,N,54000.0,Commercial associate,With parents,1,1998-09-01,Y
438555,6842765,Y,72000.0,Pensioner,House / apartment,1,1961-09-11,N


### Making connection with local database and showing schemas available(we will be inserting data to deshant_sachdeva schema)

In [32]:
with open("password.txt") as f:
    passw = f.read()

# attempt a connection
try:
    connectionString = 'mysql+mysqlconnector://root:{}@localhost:3306/deshant_sachdeva'.format(passw)
    engine = sq.create_engine(connectionString)
    connection = engine.raw_connection()

except:
    print("Problem with connection!!!")
    connection.connect()
    
schema = pd.read_sql_query("SHOW SCHEMAS;", connection)

display(schema)

,Database
0,deshant_sachdeva
1,information_schema
2,mysql
3,performance_schema
4,sys


###  Creating table application_record and columns associated with it. ID represents the primary key

In [33]:
try:

    cursor = connection.cursor()

    createTableApplicationRecord = ("""CREATE TABLE IF NOT EXISTS `application_record` (
      `ID` INT(8),
      `OWN_PROPERTY` CHAR,
      `AMT_INCOME_TOTAL` FLOAT,
      `NAME_INCOME_TYPE` VARCHAR(255),
      `NAME_HOUSING_TYPE` VARCHAR(255),
      `FLAG_MOBIL` INT(2), 
      `DATE_OF_BIRTH` DATETIME,
      `EMPLOYMENT_STATUS` CHAR,
      PRIMARY KEY (ID)
    ) ENGINE=InnoDB;""")
    
    cursor.execute(createTableApplicationRecord)
except:
    print("Problem with connection!!!")
    connection.connect()

### Inserting data into table application_record

In [34]:
application_record_data.to_sql(name="application_record", con = engine, schema="deshant_sachdeva", if_exists='replace', chunksize=10000, index=False)

print("Application record data added.")

Application record data added.


### Displaying tables

In [36]:
tables = pd.read_sql_query("SHOW TABLES;", connection)

display(tables)

,Tables_in_deshant_sachdeva
0,application_record
1,credit_record


### Displaying all of the data inserted into table application_record

In [37]:
select_all = pd.read_sql_query("""SELECT * FROM application_record;""" , connection)

display(select_all)

,ID,OWN_PROPERTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_HOUSING_TYPE,FLAG_MOBIL,DATE_OF_BIRTH,EMPLOYMENT_STATUS
0,5008804,Y,427500.0,Working,Rented apartment,1,1988-03-01,Y
1,5008805,Y,427500.0,Working,Rented apartment,1,1988-03-01,Y
2,5008806,Y,112500.0,Working,House / apartment,1,1962-03-29,Y
3,5008808,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
4,5008809,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
...,...,...,...,...,...,...,...,...
438552,6840104,Y,135000.0,Pensioner,House / apartment,1,1958-11-02,N
438553,6840222,N,103500.0,Working,House / apartment,1,1977-05-24,Y
438554,6841878,N,54000.0,Commercial associate,With parents,1,1998-09-01,Y
438555,6842765,Y,72000.0,Pensioner,House / apartment,1,1961-09-11,N


### Here in this query we are going to filter records of people that are employed and also they can be pensioners. In other ways we filter out records who have some associated income recorded. This is a necessary filter as we do not want to issue credit card to a person who can not afford to have it in first place.

In [38]:
select_with_some_income = pd.read_sql_query("""SELECT * FROM application_record where EMPLOYMENT_STATUS = 'Y'
                                                    and NAME_INCOME_TYPE <> 'Pensioner';""", connection)

display(select_with_some_income)

,ID,OWN_PROPERTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_HOUSING_TYPE,FLAG_MOBIL,DATE_OF_BIRTH,EMPLOYMENT_STATUS
0,5008804,Y,427500.0,Working,Rented apartment,1,1988-03-01,Y
1,5008805,Y,427500.0,Working,Rented apartment,1,1988-03-01,Y
2,5008806,Y,112500.0,Working,House / apartment,1,1962-03-29,Y
3,5008808,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
4,5008809,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
...,...,...,...,...,...,...,...,...
363059,6838056,Y,135000.0,State servant,House / apartment,1,1969-01-08,Y
363060,6839936,Y,135000.0,Working,House / apartment,1,1986-08-15,Y
363061,6840222,N,103500.0,Working,House / apartment,1,1977-05-24,Y
363062,6841878,N,54000.0,Commercial associate,With parents,1,1998-09-01,Y


### This query filters the records of people having some specific income and also have their own property. This query will help us figure out people who have a less probability to default on credit card payment as they might be less likely to take a home loan in near future. 

In [39]:
select_with_own_property_and_income = pd.read_sql_query("""SELECT * FROM application_record where EMPLOYMENT_STATUS = 'Y'
                                                    and NAME_INCOME_TYPE <> 'Pensioner' and OWN_PROPERTY = 'Y';""", connection)

display(select_with_own_property_and_income)

,ID,OWN_PROPERTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_HOUSING_TYPE,FLAG_MOBIL,DATE_OF_BIRTH,EMPLOYMENT_STATUS
0,5008804,Y,427500.0,Working,Rented apartment,1,1988-03-01,Y
1,5008805,Y,427500.0,Working,Rented apartment,1,1988-03-01,Y
2,5008806,Y,112500.0,Working,House / apartment,1,1962-03-29,Y
3,5008808,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
4,5008809,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
...,...,...,...,...,...,...,...,...
246811,6837905,Y,355050.0,Working,House / apartment,1,1977-06-28,Y
246812,6837906,Y,355050.0,Working,House / apartment,1,1977-06-28,Y
246813,6838056,Y,135000.0,State servant,House / apartment,1,1969-01-08,Y
246814,6839936,Y,135000.0,Working,House / apartment,1,1986-08-15,Y


### Here in this query we want to filter out people having a mobile number. Mobile number has become an important part of banking and is n=one of the methods by which we identify and notify potential credit card users and their due payments. It is highly likely that a person might miss his/her payment if they are not notified about it. Hence to decrease such risks we will be issuing a credit card to person who has a mobile number.

In [40]:
remove_people_with_no_mobileno = pd.read_sql_query("""SELECT * FROM application_record where EMPLOYMENT_STATUS = 'Y'
                                                    and NAME_INCOME_TYPE <> 'Pensioner' and OWN_PROPERTY = 'Y' and 
                                                    FLAG_MOBIL = 1;""", connection)

display(remove_people_with_no_mobileno)

,ID,OWN_PROPERTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_HOUSING_TYPE,FLAG_MOBIL,DATE_OF_BIRTH,EMPLOYMENT_STATUS
0,5008804,Y,427500.0,Working,Rented apartment,1,1988-03-01,Y
1,5008805,Y,427500.0,Working,Rented apartment,1,1988-03-01,Y
2,5008806,Y,112500.0,Working,House / apartment,1,1962-03-29,Y
3,5008808,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
4,5008809,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
...,...,...,...,...,...,...,...,...
246811,6837905,Y,355050.0,Working,House / apartment,1,1977-06-28,Y
246812,6837906,Y,355050.0,Working,House / apartment,1,1977-06-28,Y
246813,6838056,Y,135000.0,State servant,House / apartment,1,1969-01-08,Y
246814,6839936,Y,135000.0,Working,House / apartment,1,1986-08-15,Y


### This query filters the records of people having some specific income, have their own property and are living in their own house or appartment. This query will help us figure out people who will have less probability to default on credit card payment as they are less likely to spend their income to rental expenses.

In [41]:
remove_people_with_rented_apartment = pd.read_sql_query("""SELECT * FROM application_record where EMPLOYMENT_STATUS = 'Y'
                                                    and NAME_INCOME_TYPE <> 'Pensioner' and OWN_PROPERTY = 'Y' and 
                                                    FLAG_MOBIL = 1 and NAME_HOUSING_TYPE <> 'Rented apartment';""", connection)

display(remove_people_with_rented_apartment)

,ID,OWN_PROPERTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_HOUSING_TYPE,FLAG_MOBIL,DATE_OF_BIRTH,EMPLOYMENT_STATUS
0,5008806,Y,112500.0,Working,House / apartment,1,1962-03-29,Y
1,5008808,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
2,5008809,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
3,5008810,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
4,5008811,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
...,...,...,...,...,...,...,...,...
244238,6837905,Y,355050.0,Working,House / apartment,1,1977-06-28,Y
244239,6837906,Y,355050.0,Working,House / apartment,1,1977-06-28,Y
244240,6838056,Y,135000.0,State servant,House / apartment,1,1969-01-08,Y
244241,6839936,Y,135000.0,Working,House / apartment,1,1986-08-15,Y


### Here we are going to find the average of all the values from column AMT_INCOME_TOTAL. We are going to take this amount as a criteria whether a person should be given a credit card or not.

In [42]:
avg_total_income = pd.read_sql_query("""SELECT AVG(AMT_INCOME_TOTAL) FROM application_record;""", connection)

display(avg_total_income)

,AVG(AMT_INCOME_TOTAL)
0,187524.28601


### Here in this nested query we are comparing the AMT_INCOME_TOTAL and determining the amount greater then the average as a criteria to predict whether a person is eligible for credit card or not. On the individual front we are going to consider the result from this query and we will be using another dataset credit_data to strengthen the predictability criteria. We will be merging both the datasets or we will be queries from both the datasets to determine the result.

In [43]:
eligible_people_for_creditcard = pd.read_sql_query("""SELECT * FROM application_record where EMPLOYMENT_STATUS = 'Y'
                                                    and NAME_INCOME_TYPE <> 'Pensioner' and OWN_PROPERTY = 'Y' and 
                                                    FLAG_MOBIL = 1 and NAME_HOUSING_TYPE <> 'Rented apartment'
                                                    and AMT_INCOME_TOTAL > (SELECT AVG(AMT_INCOME_TOTAL) 
                                                    FROM application_record);""", connection)

display(eligible_people_for_creditcard)

,ID,OWN_PROPERTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_HOUSING_TYPE,FLAG_MOBIL,DATE_OF_BIRTH,EMPLOYMENT_STATUS
0,5008808,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
1,5008809,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
2,5008810,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
3,5008811,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
4,5008815,Y,270000.0,Working,House / apartment,1,1974-11-03,Y
...,...,...,...,...,...,...,...,...
102865,6832600,Y,315000.0,Commercial associate,House / apartment,1,1978-03-12,Y
102866,6836601,Y,337500.0,Working,House / apartment,1,1994-05-21,Y
102867,6837707,Y,202500.0,Working,House / apartment,1,1984-01-17,Y
102868,6837905,Y,355050.0,Working,House / apartment,1,1977-06-28,Y


### This query gives the total count of the records present after the filters are applied.

In [44]:
eligible_people_for_creditcard_count = pd.read_sql_query("""SELECT Count(*) FROM application_record where EMPLOYMENT_STATUS = 'Y'
                                                    and NAME_INCOME_TYPE <> 'Pensioner' and OWN_PROPERTY = 'Y' and 
                                                    FLAG_MOBIL = 1 and NAME_HOUSING_TYPE <> 'Rented apartment'
                                                    and AMT_INCOME_TOTAL > (SELECT AVG(AMT_INCOME_TOTAL) 
                                                    FROM application_record);""", connection)

display(eligible_people_for_creditcard_count)

,Count(*)
0,102870


## Loading and displaying the dataset Credit Record [[1]](https://www.kaggle.com/rikdifos/credit-card-approval-prediction)

In [13]:
credit_record = pd.read_csv('./CreditCardPrediction/credit_record.csv')

display(credit_record)

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


### SQL Queries 1.

### Cleanup and organization the dataset

### From the original dataset, each ID has different number of monthly balances, so I try to restructure the dataframe by groupby each ID. Before doing restructure, I need to check that each ID has different number of monthly balance or not? The reason why do that, if each ID has same number of monthly balance, I can restructure the dataframe by groupby() mean value of monthly balance.

In [15]:
group_by_creditrecord_id = pd.read_sql_query("""SELECT ID, COUNT(ID)
                                                FROM deshant_sachdeva.credit_record
                                                GROUP BY ID;""", connection)

display(group_by_creditrecord_id)

,ID,COUNT(ID)
0,5001711,4
1,5001712,19
2,5001713,22
3,5001714,15
4,5001715,60
...,...,...
45980,5150482,18
45981,5150483,18
45982,5150484,13
45983,5150485,2


### SQL Queries 2.

### Once I confirmed that each ID has different number of ID by the SQL. Hence, I are going to do restructure the dataframe by gourpby each ID.

In [17]:
grouping_month_id = pd.read_sql_query("""SELECT ID, MONTHS_BALANCE, STATUS
FROM  deshant_sachdeva.credit_record
GROUP BY ID, MONTHS_BALANCE;""", connection)

display(grouping_month_id)

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


### SQL Queries 3.

### Each ID has different number of monthly balances, so it's hard to make a decision the client who is credit card company approval the applicants base on the monthly balance. All ID has a credit record from our dataset, so we think correcting the data which is the most recently monthly balance's status each ID after that using an inner or outer join between two datasets and various groupings, we will structure our data in SQL. It will help to client can make a decision the application of credit card.

In [19]:
latest_month = pd.read_sql_query("""SELECT ID, MAX(MONTHS_BALANCE), STATUS
FROM  deshant_sachdeva.credit_record
GROUP BY ID;""", connection)

display(latest_month)

,ID,MAX(MONTHS_BALANCE),STATUS
0,5001711,0,X
1,5001712,0,C
2,5001713,0,X
3,5001714,0,X
4,5001715,0,X
...,...,...,...
45980,5150482,-11,C
45981,5150483,0,X
45982,5150484,0,C
45983,5150485,0,0


### SQL Queries 4.

### If applicants complain that it is unfair reject or approval by only last month of balances, so we should think other ways. In other way, we can drop the data that applicants who have any overdue or bad debts by their credit status. If applicants have any overdue or bad debts before, the client will reject their credit card application. Based on the dataset, starting from 2 is that overdue or bad debts at STATUS. For example, the value of STATUS is over 1.5, applicants will be rejected their application.

In [20]:
filter_applications = pd.read_sql_query("""SELECT ID, MONTHS_BALANCE, STATUS
FROM deshant_sachdeva.credit_record
WHERE STATUS not like '2' and '3' and '4' and '5'
GROUP BY ID;""", connection)

display(filter_applications)
# citation from W3 Schools - (6)

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001712,0,C
2,5001713,0,X
3,5001714,0,X
4,5001715,0,X
...,...,...,...
45980,5150482,-11,C
45981,5150483,0,X
45982,5150484,0,C
45983,5150485,0,0


### SQL Queries 5.

### Moreover, each ID has different number of monthly balances, so we can calculate the average of STATUS and we set up the criteria of average of STATUS. If applicants who have over our criteria of average of STATUS, applicants will get reject their credit card application.

In [25]:
avg_status = pd.read_sql_query("""SELECT ID, MONTHS_BALANCE, AVG(STATUS)
FROM deshant_sachdeva.credit_record
WHERE STATUS
GROUP BY ID;""", connection)

display(avg_status)
# citation from W3 Schools - (7)

,ID,MONTHS_BALANCE,AVG(STATUS)
0,5001718,-8,1.0
1,5001720,0,1.0
2,5001757,-3,1.0
3,5001786,-38,1.0
4,5001792,-2,1.0
...,...,...,...
5345,5150412,-7,1.0
5346,5150417,-30,1.0
5347,5150459,-39,1.0
5348,5150463,0,1.0


We restructured the dataset are 3 ways that we will merge together with our other datasets to showing which ways are the best fit for our dataset.

It is informative. The reason why, one of important the primary techniques of a data scientist is data wrangling. On the real-world problem, there has so many different types of ways to data wrangling at the same problem, so I tried to different ways of data wrangling for using an inner or outer join between two datasets and various groupings, we will structure our data in SQL. I learnt that whatever platform I finish up using the ability to think about what types of dataset I need and working on with that dataset is a highly relevant and data wrangling skill from this project.

It's biased because this dataset is from kaggle.com and author is Seanny whose Student at University of Maryland in United States. From the website, 9 reviewers had the opportunity to review this material and the dataset got 10/10 Usability score which means that Easy to understand and includes essential data, Rich, machine readable file formats and data and Assurances the dataset is maintained. Also, most of 9 reviewers are Data Scientist or Data Science Students. Hence, I would like to say that I trust the reviewers, so it is rigorous.(1)

There are indicators about whether the information is accurate. The author replied the someone's comments for asking data source. He mentioned that the dataset was originally from Chinese website. It has somebody else whose teacher of online data science course instruction in China written about this dataset by R Studio.(6) I think it is correct dataset. Also, there has some errors about understanding the monthly balances that shows negative values. Some of reviewers provided their own critique the monthly balances to understand the value of monthly balances. It is useful because I couldn’t understand why the monthly balances are negative value as well.

Organizing our dataset to serve a particular need is another skill that is relevant both in python and SQL and across platforms. This inidividual milestone helps me apply that skill on a real-world problem.

## Final merging

### Credit record final data

In [27]:
credit_records_final_data = pd.read_sql_query("""SELECT DISTINCT(ID), STATUS
FROM deshant_sachdeva.credit_record
WHERE (STATUS = 'C' or STATUS = 'X') and MONTHS_BALANCE = 0
GROUP BY ID;""", connection)

display(credit_records_final_data)

,ID,STATUS
0,5001711,X
1,5001712,C
2,5001713,X
3,5001714,X
4,5001715,X
...,...,...
24533,5150479,C
24534,5150481,X
24535,5150483,X
24536,5150484,C


### Merging Application record and Credit record dataset

#### Credit record Final Data

In [45]:
credit_records_final_data = pd.read_sql_query("""SELECT DISTINCT(ID), STATUS
FROM deshant_sachdeva.credit_record
WHERE (STATUS = 'C' or STATUS = 'X') and MONTHS_BALANCE = 0
GROUP BY ID;""", connection)

display(credit_records_final_data)

,ID,STATUS
0,5001711,X
1,5001712,C
2,5001713,X
3,5001714,X
4,5001715,X
...,...,...
24533,5150479,C
24534,5150481,X
24535,5150483,X
24536,5150484,C


#### Application record Final Data

In [46]:
eligible_people_for_creditcard = pd.read_sql_query("""SELECT * FROM application_record where EMPLOYMENT_STATUS = 'Y'
                                                    and NAME_INCOME_TYPE <> 'Pensioner' and OWN_PROPERTY = 'Y' and 
                                                    FLAG_MOBIL = 1 and NAME_HOUSING_TYPE <> 'Rented apartment'
                                                    and AMT_INCOME_TOTAL > (SELECT AVG(AMT_INCOME_TOTAL) 
                                                    FROM application_record);""", connection)

display(eligible_people_for_creditcard)

,ID,OWN_PROPERTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_HOUSING_TYPE,FLAG_MOBIL,DATE_OF_BIRTH,EMPLOYMENT_STATUS
0,5008808,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
1,5008809,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
2,5008810,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
3,5008811,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
4,5008815,Y,270000.0,Working,House / apartment,1,1974-11-03,Y
...,...,...,...,...,...,...,...,...
102865,6832600,Y,315000.0,Commercial associate,House / apartment,1,1978-03-12,Y
102866,6836601,Y,337500.0,Working,House / apartment,1,1994-05-21,Y
102867,6837707,Y,202500.0,Working,House / apartment,1,1984-01-17,Y
102868,6837905,Y,355050.0,Working,House / apartment,1,1977-06-28,Y


### Final Result

In [50]:
final_result_set = pd.read_sql_query("""SELECT app_rec.ID, credit_rec.STATUS, app_rec.OWN_PROPERTY, app_rec.AMT_INCOME_TOTAL, 
app_rec.NAME_INCOME_TYPE, NAME_HOUSING_TYPE, app_rec.FLAG_MOBIL, 
app_rec.DATE_OF_BIRTH, app_rec.EMPLOYMENT_STATUS
FROM deshant_sachdeva.application_record as 
app_rec

INNER JOIN 

deshant_sachdeva.credit_record as credit_rec
on app_rec.ID = credit_rec.ID

where app_rec.EMPLOYMENT_STATUS = 'Y' and 
app_rec.NAME_INCOME_TYPE <> 'Pensioner' and app_rec.OWN_PROPERTY = 'Y' and 
app_rec.FLAG_MOBIL = 1 and app_rec.NAME_HOUSING_TYPE <> 'Rented apartment'
and 
app_rec.AMT_INCOME_TOTAL > (SELECT AVG(AMT_INCOME_TOTAL) FROM deshant_sachdeva.application_record) 
and 
(credit_rec.STATUS = 'C' or credit_rec.STATUS = 'X') and 
credit_rec.MONTHS_BALANCE = 0
GROUP BY credit_rec.ID;""", connection)

display(final_result_set)

,ID,STATUS,OWN_PROPERTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_HOUSING_TYPE,FLAG_MOBIL,DATE_OF_BIRTH,EMPLOYMENT_STATUS
0,5008810,C,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
1,5008811,C,Y,270000.0,Commercial associate,House / apartment,1,1968-09-17,Y
2,5008839,C,Y,405000.0,Commercial associate,House / apartment,1,1988-08-11,Y
3,5008840,C,Y,405000.0,Commercial associate,House / apartment,1,1988-08-11,Y
4,5008843,C,Y,405000.0,Commercial associate,House / apartment,1,1988-08-11,Y
...,...,...,...,...,...,...,...,...,...
3838,5135991,C,Y,585000.0,Commercial associate,House / apartment,1,1992-06-23,Y
3839,5142185,C,Y,450000.0,Commercial associate,House / apartment,1,1972-03-09,Y
3840,5145846,C,Y,256500.0,State servant,House / apartment,1,1972-11-11,Y
3841,5149145,C,Y,247500.0,Working,House / apartment,1,1991-01-18,Y


#### Final Count

In [49]:
final_count = pd.read_sql_query("""SELECT count(*)
FROM deshant_sachdeva.application_record as 
app_rec

INNER JOIN 

deshant_sachdeva.credit_record as credit_rec
on app_rec.ID = credit_rec.ID

where app_rec.EMPLOYMENT_STATUS = 'Y' and 
app_rec.NAME_INCOME_TYPE <> 'Pensioner' and app_rec.OWN_PROPERTY = 'Y' and 
app_rec.FLAG_MOBIL = 1 and app_rec.NAME_HOUSING_TYPE <> 'Rented apartment'
and app_rec.AMT_INCOME_TOTAL > (SELECT AVG(AMT_INCOME_TOTAL) 
FROM deshant_sachdeva.application_record) and 
(credit_rec.STATUS = 'C' or credit_rec.STATUS = 'X') and 
credit_rec.MONTHS_BALANCE = 0;""", connection)

display(final_count)

,count(*)
0,3843


### References

- 1 Dataset, https://www.kaggle.com/rikdifos/credit-card-approval-prediction, Source: https://www.kaggle.com

- 2 Information, https://stackoverflow.com/questions/40104946/how-to-get-date-after-subtracting-days-in-pandas, Source: https://stackoverflow.com

- 3.Seanny. (2020, March 24). Credit Card Approval Prediction. Kaggle. https://www.kaggle.com/rikdifos/credit-card-approval-prediction?select=credit_record.csv

- 4.Seanny. (2020, March 24). Credit Card Approval Prediction. Kaggle. https://www.kaggle.com/rikdifos/credit-card-approval-prediction?select=application_record.csv

- 5.Davies, C. (2021, November 15). MYSQL Connector. University of Calgary.
https://d2l.ucalgary.ca/d2l/le/content/400972/viewContent/4998206/View

- 6.SQL LIKE Operator. (n.d.). W3 Schools. Retrieved November 22, 2021, from https://www.w3schools.com/sql/sql_like.asp

- 7.SQL COUNT(), AVG() and SUM() Functions. (n.d.). W3 Schools. Retrieved November 22, 2021, from https://www.w3schools.com/sql/sql_count_avg_sum.asp

- 8.Xiong, X. (2019, October 22). 熊学堂 · 课程介绍 | 在线实习 之 《信用卡申请评分模型》. 微信公众平台. https://mp.weixin.qq.com/s/upjzuPg5AMIDsGxlpqnoCg